In [ ]:
from data_collection import (store_database_for_eys_gene,
                             parse_lovd,
                             LOVD_PATH,
                             set_lovd_dtypes)

In [ ]:
store_database_for_eys_gene("lovd", override=False)

In [ ]:
data = parse_lovd(LOVD_PATH + "/lovd_data.txt")
for i in data:
    print(i)
    display(data[i])

In [ ]:
set_lovd_dtypes(data)
for i in data:
    print(i)
    display(data[i].info())

In [ ]:
from tools import (fetch_cadd_score, fetch_cadd_scores)
import pandas as pd

row_idx = 1
selected_row = data["Variants_On_Genome"].iloc[row_idx]

selected_row_df = pd.DataFrame([selected_row])
headers = pd.DataFrame(columns=selected_row.index)
result_df = pd.concat([headers, selected_row_df])
display(result_df)

chromosome = selected_row["chromosome"]
pos_start = selected_row["position_g_start"]
pos_end = selected_row["position_g_end"]

display("Variant scoring:")
score = fetch_cadd_scores("GRCh38-v1.7", chromosome, pos_start, pos_end)
score_df = pd.DataFrame(score[1:], columns=score[0])
display(score_df)
      